# Performing a Sky Simulation with RFI Sources

This example shows how to include RFI into the simulation. We take an arbitrary sky and use karabo to simulate the visibilities and the interference due to satellites.

In [ ]:
import os
from karabo.simulation.observation import Observation
from karabo.simulation.sky_model import SkyModel
from karabo.simulation.telescope import Telescope, VLAVersions
from karabo.simulation.beam import generate_gaussian_beam_data
from karabo.simulation.interferometer import FilterUnits, InterferometerSimulation
from rascil.processing_components import create_visibility_from_ms
from karabo.simulator_backend import SimulatorBackend
from karabo.util.file_handler import FileHandler
from karabo.simulation.signal.rfi_signal import RFISignal
from pathlib import Path

import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u
from datetime import datetime, timedelta

from karabo.imaging.imager_wsclean import WscleanDirtyImager, WscleanDirtyImagerConfig

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
obs_vla = {
    "tel_name": "vla",
    "fov": 4.0,
    "phase_centre": [100, 34],
    "start_time": datetime(2023, 2, 22, 4, 12, 51),
    "obs_length": timedelta(hours=0, minutes=0, seconds=40, milliseconds=0),
    "int_time": 2.0,
    "chan_width": 1.0e6,
    "n_freq": 1,    
    # "start_freq": 5e6, # Starlink far-field
    "start_freq": 15e6, # Starlink far-field
    # "start_freq": 45e6, # Starlink near-field
    "rfi_flux_scale": 10e-9,
    # "rfi_flux_scale": 5e-9,
    # "sat_names": ["starlink"],
    # "norad_ids": [],
    "sat_names": [],
    "norad_ids": [46038],
    "max_n_sats": 1,
    "min_altitude_deg": 0.0, # Above the horizon
    "max_angular_sep_deg": 80.0,
}

mkat_pc = SkyCoord(ra="15:16:40.2", dec="+00:15:02.0", unit=(u.hourangle, u.deg))

obs_mkat = {
    "tel_name": "meerkat",
    "fov": 4.0,
    "phase_centre": [float(mkat_pc.ra.deg), float(mkat_pc.dec.deg)],
    "start_time": datetime(2019, 5, 13, 23, 25, 15),
    "obs_length": timedelta(seconds=896),
    "int_time": 8.0,
    "chan_width": 1.0e6,
    "n_freq": 1,    
    # "start_freq": 5e6, # Galileo far-field
    "start_freq": 15e6, # Galileo far-field
    # "start_freq": 45e6, # Galileo near-field
    "rfi_flux_scale": 1e-5,
    # "rfi_flux_scale": 5e-5,
    "sat_names": [],
    "norad_ids": [40544],
    # "sat_names": ["galileo"],
    # "norad_ids": [],
    "max_n_sats": None,
    "min_altitude_deg": 0.0, # Above the horizon
    "max_angular_sep_deg": 40.0,
}

In [ ]:
obs = obs_vla
# obs = obs_mkat

obs["n_time"] = int(obs["obs_length"].total_seconds() / obs["int_time"])

In [ ]:
# grid_srcs = False
grid_srcs = True

# Read in and plot the sky model
# sky = SkyModel.sky_test()
if grid_srcs:
    ras = obs["phase_centre"][0] + np.linspace(-obs["fov"]/2, obs["fov"]/2, 5)
    decs = obs["phase_centre"][1] + np.linspace(-obs["fov"]/2, obs["fov"]/2, 5)
    ra_arr, dec_arr = np.meshgrid(ras, decs)
    
    sky = SkyModel()
    sky_data = np.zeros((ra_arr.size, SkyModel.SOURCES_COLS))
    sky_data[:, 0] = ra_arr.flatten()
    sky_data[:, 1] = dec_arr.flatten()
    sky_data[:, 2] = 1
    sky.add_point_sources(sky_data)
else:
    # Sky Model if you want ONE POINT SOURCE ONLY
    sky = SkyModel()
    sky_data = np.array([[*obs["phase_centre"], 1]])
    sky.add_point_sources(sky_data)

sky.explore_sky(phase_center=obs["phase_centre"], wcs_enabled=True, cfun=lambda x: x, cbar_label="Flux [Jy]")

In [ ]:
# Simulation settings that are the same for OSKAR and RASCIL
observation = Observation(
    phase_centre_ra_deg=obs["phase_centre"][0],
    phase_centre_dec_deg=obs["phase_centre"][1],
    start_date_and_time=obs["start_time"],
    length=obs["obs_length"],
    number_of_time_steps=obs["n_time"],
    start_frequency_hz=obs["start_freq"],
    frequency_increment_hz=obs["chan_width"],
    number_of_channels=obs["n_freq"],
)

In [ ]:
# `sim-vis` which is used for the RFI simulation relies on data from SpaceTrack service.
# It needs a login with username and password. Thus, we must provide the login as
# a .yaml file.

credentials_filename = "my_test_credentials.yaml"
# with open(credentials_filename, "w") as f:
#     f.write("username: <add your username>\n")
#     f.write("password: <add ypur password>\n")


In [ ]:
# The class `RFISignal` controls the simulation of RFI sources
rfi_signal = RFISignal(credentials_filename)
rfi_signal.set_rfi_flux_scale(obs["rfi_flux_scale"])
rfi_signal.set_satellites(sat_names=obs["sat_names"], norad_ids=obs["norad_ids"])
rfi_signal.filter_satellites(max_n_sats=obs["max_n_sats"], min_altitude_deg=obs["min_altitude_deg"], max_angular_sep_deg=obs["max_angular_sep_deg"])

In [ ]:
simulation = InterferometerSimulation(
    channel_bandwidth_hz=obs["chan_width"],
    time_average_sec=obs["int_time"],
    ignore_w_components=False,
    uv_filter_units=FilterUnits.Metres,
    use_gpus=True,
    use_dask=False,
    station_type="Isotropic beam",  # This is only used by OSKAR (beam setting in OSKAR)
)

In [ ]:
output_base_directory = Path(
    FileHandler().get_tmp_dir(
        prefix="rfi-contaminated-",
        purpose="Example satellite RFI simulation",
    )
)

In [ ]:
#### OSKAR SIMULATION
# Load the telescope (VLA D configuration [compact - max 2 km])
if obs["tel_name"]=="vla":
    telescope = Telescope.constructor("VLA", VLAVersions.D, backend=SimulatorBackend.OSKAR)
else:
    telescope = Telescope.constructor("MeerKAT", backend=SimulatorBackend.OSKAR)
    # telescope = Telescope.constructor("EDA2", backend=SimulatorBackend.OSKAR)

visibility = simulation.run_simulation(
    telescope=telescope,
    sky=sky,
    observation=observation,
    backend=SimulatorBackend.OSKAR,
    primary_beam=None,
    visibility_format="MS",
    visibility_path=os.path.join(output_base_directory, "OSKAR.MS"),
    rfi_signals=rfi_signal
)

In [ ]:
ang_min = np.min(rfi_signal.xds.rfi_tle_sat_ang_sep.data, axis=(1,2)).compute()
np.argmin(ang_min), np.min(ang_min)

In [ ]:
xyz = (
    rfi_signal.xds.rfi_tle_sat_xyz.data[:, :: rfi_signal.xds.n_int_samples]
    - rfi_signal.xds.ants_xyz.data.mean(axis=1)[None, :: rfi_signal.xds.n_int_samples, :]
)
d_f = 2 * telescope.max_baseline()**2 / (3e8 / obs["start_freq"])
R = np.linalg.norm(xyz[0], axis=-1).compute()
print(f"Average satellite distance   : {R.mean()/1e3:.0f} km")
print(f"Near-field boundary distance : {d_f/1e3:.0f} km")

In [ ]:
# Set Imaging parameters
if obs["tel_name"]=="vla":
    fov = np.deg2rad(35)
else:
    fov = np.deg2rad(15)
imaging_cellsize = np.deg2rad(telescope.ang_res(obs["start_freq"], telescope.max_baseline())/3600 / 5)
imaging_npixel = 2 * int(fov / imaging_cellsize / 2)
print(f"Image size : ({imaging_npixel}x{imaging_npixel})")


In [ ]:
# Imaging with WSClean
imager_single = WscleanDirtyImager(
    WscleanDirtyImagerConfig(
        imaging_npixel=imaging_npixel,
        imaging_cellsize=imaging_cellsize,
    )
)

dirty_image = imager_single.create_dirty_image(visibility)
img_fig = dirty_image.plot(title="Dirty image WSClean")

In [ ]:
sat_pos = rfi_signal.get_satellite_radec()
ax = img_fig.axes[0]                 # or grab the specific WCSAxes you want
world = ax.get_transform('world')  # 'world' = RA/Dec in the axis' native frame
ax.plot(sat_pos.ra, sat_pos.dec, 'o', alpha=1, ms=6, mfc='none', mec='black', transform=world) 

img_fig


In [ ]:
imager_multi = WscleanDirtyImager(
    WscleanDirtyImagerConfig(
        imaging_npixel=imaging_npixel,
        imaging_cellsize=imaging_cellsize,
        intervals_out=obs["n_time"],
    )
)

imager_multi.create_dirty_image_series(visibility)
anim = imager_multi.animate(
    # interval_ms=200,
    interval_ms=50,
    normalize="global",         # or "per_frame"
    samples_per_frame=5000,
    markers_world_per_frame=sat_pos,
    marker_radius_deg=2.0,
)